In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os, sys, email
import gensim
from gensim.models import Doc2Vec
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from string import punctuation
import timeit
from sklearn.cluster import KMeans
from sklearn import metrics
import pylab as pl
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
start = timeit.default_timer()
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
from smart_open import smart_open
from random import shuffle

pd.set_option('display.max_colwidth',1000)

In [2]:
# Load dataset
data = pd.read_csv('processed_resumes_work_ADDED_JOB_TITLES.csv')

In [5]:
# Removed jobs without description
data = data[~data.descript.isnull()]

# Convert null job title into blank
data.loc[data.role.isnull()] = ""

In [7]:
# Create dataset for training doc2vec
data['train_val'] = data.role + " " + data.descript.str.strip()

In [8]:
data['train_val'].count()

1229127

In [9]:
data['train_val'].nunique()

1214940

In [10]:
def clean_data(data):
    stop_words = set(stopwords.words('english'))
    lmtzr = WordNetLemmatizer()

    cleaned_list = []
    
    for row in data:
        row = re.sub(r'[^a-zA-Z ]+', ' ', row.lower())
        row = ' '.join([lmtzr.lemmatize(word) for word in row.split() if word not in stop_words])
        cleaned_list.append(row)    
    
    return cleaned_list    

In [25]:
original_data[328412]

'application architect .TOKYO, JAPAN\xa0.ING L fe I su ce Comp y (Jp)Lm ted p ov des fe su ce p oducts d se v ces\xa0.fo E ow e s. ING L Po t (INGLP)sfudmet system povdgv ous\xa0.se v ces of the ING fe su ce to Age t, R, Custome d ING emp oyees. Th s INGLP s\xa0.etypot the web whe e ING c p omote ew p oducts d se v ces, teg te\xa0.d ffe e t web ppctosfop ocess gdhdg custome tsctos, docume t\xa0.p oduct gudeesdp ov de st uct o s d educ to cotetfoRsd ge ts..'

In [24]:
cleaned_data.index("application architect tokyo japan ing l fe su ce comp jp lm ted p ov de fe su ce p oducts se v ce fo e ow e ing l po inglp sfudmet system povdgv ous se v ce ing fe su ce age r custome ing emp oyees th inglp etypot web whe e ing c p omote ew p oducts se v ce teg te ffe e web ppctosfop ocess gdhdg custome tsctos docume p oduct gudeesdp ov de st uct educ cotetforsd ge t")

328412

In [23]:
[s for s in cleaned_data if "application architect tokyo japan" in s]

['application architect tokyo japan ing l fe su ce comp jp lm ted p ov de fe su ce p oducts se v ce fo e ow e ing l po inglp sfudmet system povdgv ous se v ce ing fe su ce age r custome ing emp oyees th inglp etypot web whe e ing c p omote ew p oducts se v ce teg te ffe e web ppctosfop ocess gdhdg custome tsctos docume p oduct gudeesdp ov de st uct educ cotetforsd ge t']

In [11]:
original_data = data.train_val.unique()

In [20]:
# cleaned_data = clean_data(original_data)

In [18]:
import pickle

# with open('cleaned_data', 'wb') as fp:
#     pickle.dump(cleaned_data, fp)

In [19]:
cleaned_data = pickle.load(open('cleaned_data', 'rb'))

In [ ]:
len(traindocs)

In [468]:
# Currently testing model process on 100,000 records
# 95k for training set, 5k for test set
traindocs = []
testdocs = []
for index, line in enumerate(cleaned_data):
    words = gensim.utils.to_unicode(line).split()
    split = ['train', 'test'][index//1200000]
    
    if split == 'train':
        traindocs.append(TaggedDocument(words, [index]))
    else:
        testdocs.append(TaggedDocument(words, [index]))

print('%d train, %d test' % (len(traindocs), len(testdocs)))

1200000 train, 14940 test


In [470]:
with open('traindocs', 'wb') as fp:
    pickle.dump(traindocs, fp)

In [471]:
with open('testdocs', 'wb') as fp:
    pickle.dump(testdocs, fp)

In [156]:
model = gensim.models.Doc2Vec(traindocs, dm = 1, dbow_words = 1, 
                              negative = 5, alpha=0.025, size= 200, 
                              min_alpha=0.025, min_count=0
                             )
 
model.train(traindocs, total_examples=1, epochs=1)

/Users/kwheatley/anaconda/envs/python36/lib/python3.6/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [173]:
model.most_similar(['actual'])
# model['ceo']
# model.docvecs.most_similar(2)
# traindocs[2]

/Users/kwheatley/anaconda/envs/python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('actuals', 0.487826406955719),
 ('versus', 0.4535791575908661),
 ('forecasted', 0.40754592418670654),
 ('expected', 0.3885841965675354),
 ('overrun', 0.386729896068573),
 ('compared', 0.37691235542297363),
 ('reasonableness', 0.3718114197254181),
 ('unexpected', 0.3611285090446472),
 ('baseline', 0.3575695753097534),
 ('projected', 0.35488003492355347)]

In [253]:
# len(traindocs)
# model.docvecs[85916]
traindocs[85917]

TaggedDocument(words=['designer', 'bim', 'cad', 'manager', 'design', 'permitting', 'construction', 'oversight', 'single', 'multi', 'family', 'residential', 'commercial', 'retail', 'mixed', 'use', 'educational', 'facility', 'design', 'conducted', 'urban', 'design', 'planning', 'ordinance', 'design', 'well', 'web', 'graphic', 'design', 'coordinated', 'team', 'engineer', 'architect', 'construction', 'manager', 'design', 'project', 'w', 'construction', 'cost', 'ranging', 'k', 'activity', 'included', 'contract', 'negotiation', 'zoning', 'ordinance', 'review', 'building', 'code', 'research', 'review', 'construction', 'document', 'design', 'permit', 'expediting', 'interfaced', 'managed', 'coordination', 'client', 'consultant', 'vendor', 'throughout', 'phase', 'design', 'implemented', 'maintained', 'computer', 'system', 'server', 'software', 'set', 'managed', 'emerging', 'standard', 'bim', 'cad', 'software', 'revit', 'acad', 'designed', 'maintained', 'urban', 'studio', 'inc', 'company', 'websi

In [260]:
# model.docvecs[94999]

In [266]:
# filevector = open('docvectors.txt', 'w')
filemeta = open('docmeta.txt', 'w')
# filemeta.write("%s\n" % 'title')

for index in range(95000):
#     cleaned_vectors = '\t'.join(str(vector) for vector in model.docvecs[index])
#     filevector.write("%s\n" % cleaned_vectors)
    filemeta.write("%s\n" % (str(index) + ' ' + ' '.join(traindocs[index].words[0:5])))

In [206]:
traindocs[3847]

TaggedDocument(words=['national', 'account', 'manager', 'accountable', 'managing', 'client', 'portfolio', 'client', 'totaling', 'individual', 'location', 'hospitality', 'retail', 'travel', 'vertical', 'total', 'annual', 'recurring', 'revenue', 'million', 'responsible', 'achieving', 'non', 'recurring', 'revenue', 'profit', 'margin', 'monthly', 'support', 'revenue', 'quota', 'constantly', 'exceeded', 'goal', 'consulted', 'sold', 'maintained', 'guest', 'back', 'house', 'wireless', 'infrastructure', 'named', 'national', 'account', 'resulting', 'trusted', 'advisor', 'role', 'formulated', 'strategic', 'sale', 'plan', 'stimulated', 'account', 'growth', 'retention', 'executed', 'creation', 'presentation', 'proposal', 'contract', 'monthly', 'reporting', 'portal', 'rebranding', 'marketing', 'material', 'utilized', 'salesforce', 'tm', 'sale', 'activity', 'hardware', 'distribution', 'customized', 'reporting', 'used', 'strong', 'project', 'management', 'skill', 'organize', 'secure', 'manage', 'comp

In [465]:
len(traindocs)

95000

In [286]:
# Let's look at the data more. Are the majority of these jobs entry level or higher level?
# We will base it on their graduation date.

# Load dataset
edu_data = pd.read_csv('processed_resumes_education.csv',header=None)

In [287]:
edu_data.columns = ['city','resume_id','container','degree','school','location','dates']
edu_data.head()


,city,resume_id,container,degree,school,location,dates
0,atlanta,0004d469fc497102,education-items,Bachelor of Science in Computer Science,Wichita State University,"Wichita, KS",1990 to 1994
1,atlanta,0006216bf673dc16,education-items,NaN,Youth Academy Soccer Coach,NaN,2018
2,atlanta,0006216bf673dc16,education-items,Bachelor of Arts in History,Georgia State University,"Atlanta, GA",August 2004
3,atlanta,0006216bf673dc16,education-items,NaN,Wake Forest University,NaN,1995 to 1997
4,atlanta,000cb5da9472be95,education-items,Bachelor of Science in Human Resources Management,Louisiana State University,NaN,NaN


In [305]:
edu_data[edu_data.degree == 'n economics']

,city,resume_id,container,degree,school,location,dates,degree_changed


In [357]:
edu_data['degree_changed'] = edu_data.degree.str.strip()
edu_data = edu_data[~edu_data.degree_changed.isnull()]
edu_data = edu_data[edu_data.degree_changed != '']

In [362]:
edu_data[edu_data.degree_changed == '']

,city,resume_id,container,degree,school,location,dates,degree_changed


In [367]:
sorted_data = edu_data.degree_changed.sort_values().unique()

In [370]:
sorted_data[145508]

'交換留学 in 応用数学と統計学'

In [368]:
edu_cleaned_data = clean_data(sorted_data)

In [371]:
edu_cleaned_data = [i for i in edu_cleaned_data if i != '']

In [380]:
len(edu_cleaned_data)

145451

In [381]:
# Currently testing model process on 100,000 records
# 95k for training set, 5k for test set
edu_traindocs = []
edu_testdocs = []
for index, line in enumerate(edu_cleaned_data):
    words = gensim.utils.to_unicode(line).split()
    edu_traindocs.append(TaggedDocument(words, [index]))

In [439]:
model = gensim.models.Doc2Vec(edu_traindocs, dm = 0, dbow_words = 1, 
                              negative = 5, alpha=0.025, size= 100, 
                              min_alpha=0.025, min_count=50
                             )
 
model.train(edu_traindocs, total_examples=1, epochs=50)

/Users/kwheatley/anaconda/envs/python36/lib/python3.6/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [456]:
len(model.docvecs)

145451

In [459]:
# edu_filevector = open('edu_docvectors.txt', 'w')
edu_meta_list = []
edu_filemeta = open('edu_docmeta.txt', 'w')
# edu_filemeta.write("%s\n" % 'title')

for index in range(len(edu_traindocs)):
    cleaned_vectors = '\t'.join(str(vector) for vector in model.docvecs[index])
    if not cleaned_vectors or cleaned_vectors == '':
        print("issue with vector")
        print(index)
    edu_filevector.write("%s\n" % cleaned_vectors)
    cleaned_meta = (str(index) + ' ' + ' '.join(edu_traindocs[index].words))
#     if index >= 94999:
#         print(index)
    if not cleaned_meta:
        print("issue with meta")
        print(index)
    edu_filemeta.write("%s\n" %cleaned_meta )

In [440]:
# model.most_similar(['actual'])
# model['ceo']
model.docvecs.most_similar(5367)
# edu_traindocs[5367]

[(551, 0.8868790864944458),
 (70196, 0.878495454788208),
 (2000, 0.8709387183189392),
 (80538, 0.8597651124000549),
 (70195, 0.851523756980896),
 (107155, 0.8514955639839172),
 (59046, 0.8435146808624268),
 (77950, 0.8270519971847534),
 (63751, 0.8169573545455933),
 (113529, 0.8154358863830566)]

In [429]:
model.docvecs.most_similar(5367)

[(551, 0.9420228004455566),
 (107155, 0.9348306059837341),
 (59046, 0.9313094019889832),
 (113529, 0.9298691153526306),
 (2000, 0.9274170398712158),
 (70195, 0.9271657466888428),
 (70196, 0.9147785902023315),
 (38485, 0.9115675091743469),
 (63751, 0.9100584387779236),
 (80538, 0.9061126112937927)]

In [441]:
edu_traindocs[551]

TaggedDocument(words=['business', 'administrator'], tags=[551])

In [442]:
edu_traindocs[70196]

TaggedDocument(words=['business', 'administrator', 'business'], tags=[70196])

In [444]:
edu_traindocs[113529]

TaggedDocument(words=['master', 'business', 'administrator'], tags=[113529])

In [ ]:
# After we get the clustering of documents, we can try to derive the topic
# https://stackoverflow.com/questions/46047506/how-to-find-most-similar-terms-words-of-a-document-in-doc2vec?rq=1